On veut simuler un salon de coiffure avec les données suivantes:

Un seul coiffeur
La moyenne d'arrivé des clients est de 5 clients par heure
La durée moyenne d'une coupe est de 10 minutes
La salle d'attente comporte 4 places
Le prix d'une coupe est de 500da
1- Estimez par simulation sur 14.400 minutes (sans afficher les détails de chaque arrivée) les métriques suivantes:

Nombre de clients reçus
Nombre de clients traités
Taux de rejet(%)
Temps d'attente moyen d'un client dans la salle d'attente
La charge de travail du coiffeur
Le chiffre d'affaire total
2- Que deviennent ces statistiques pour une salle d'attente de capacité = 6? est-ce intéressant d'augmenter le nombre de places? argumentez.

3- Est il intéressant pour le salon de recruter un autre coiffeur avec un salaire mensuel de 70.000da? argumentez après avoir estimé les métriques de la q1

4- Est il intéressant pour le salon de recruter 2 coiffeurs avec un salaire mensuel de 70.000da chacun? argumentez après avoir estimé les métriques de la q1

source : Exercice 02 fiche TP 03 MODSIM M1 USTO-MB 

In [1]:
import scipy.stats as st
import numpy as np
import simpy

nb_rejet = 0
t_attente =[]
temps_coupe=[]
id_client = 0

####################################################################################

# calcul des métriques
def metriques(nom_coiffeurs,duree,salaire,prix):
    global id_client
    global nb_rejet
    global t_attente
    global temps_coupe
    print ("nombre de clients reçus: ",id_client+1)
    print ("nombre de clients rejetés: ",nb_rejet)
    print ("nombre de clients traités: ",id_client+1-nb_rejet)
    print ("taux de rejet: %.2f"%(nb_rejet*100/(id_client+1)),"%")
    print ("temps d'attente moyen: ",sum(t_attente)/len(t_attente))
    #Charge du coiffeur i % = Somme des temps de coupe du coiffeur i *100 / la durée de la simulation
    for i in range (len(nom_coiffeurs)):
        # Charge du coiffeur i % = Somme des temps de coupe du coiffeur i *100 / la durée de la simulation
        print ("charge du %s : %.2f"%(nom_coiffeurs[i],temps_coupe[i]*100/duree))
    if prix>0 :
        print ("Chiffre d'affaires total: ",(id_client+1-nb_rejet)*prix,"DA")
    if salaire>0 :
        print ("Chiffre d'affaires final (après payement des salaires)",(id_client+1-nb_rejet)*500-salaire*len(nom_coiffeurs),"DA")

###################################################################################

# fonction de simulation
# duree: la durée de la simulation
# moy_client: nombre de clients moyen par unitée de temps ( ici minute )
# moy_coupe: durée moyenne pour qu'un coiffeur termine une coupe
# nom_coiffeurs: la liste des noms des coiffeurs
# capacité: la capacité maximale de la file d'attente

def simulation(duree, moy_client, moy_coupe, nom_coiffeurs,capacite, affichage):
    
    nb_coiffeurs=len(nom_coiffeurs) # le nombre de serveurs
    
    # initialiser la liste du temps de service de chaque serveur
    for i in range (nb_coiffeurs) :
        temps_coupe.append(0)
        
    file=[] #créer une file d'attente vide
    env = simpy.Environment()  #création de l'environnement de simulation
    
    coiffeurs=simpy.Resource(env, capacity=nb_coiffeurs) #déclaration d'une ressource partagée (ici les serveurs)
    
    #déclaration du processus principal
    env.process(generation_clients(env, moy_client, moy_coupe, coiffeurs, nom_coiffeurs, file, capacite, affichage)) 
    print("\n[%.2f] -------------- Début de la simulation --------------"%(env.now))
    env.run(until=duree) #lancement de la simulation pendant une duree
    print("[%.2f] -------------- Fin de la simulation ----------------\n"%(env.now))
        
###################################################################################

def generation_clients(env, moy_client, moy_coupe, coiffeurs, nom_coiffeurs, file, capacite, affichage):
    global id_client
    global nb_rejet
    id_client = 0
    nb_rejet = 0
    while True:
        yield env.timeout(1) #avancer l'horloge du simulateur de 1 unité de temps
        nbrclients=st.poisson(moy_client).rvs() #simuler le nombre de clients par la loi de poisson
        for cl in range(nbrclients): #générer un processus de traitement pour chaque client
            #création du processus qui traite ce client
            env.process(traitement_client(env, moy_coupe, coiffeurs, id_client, nom_coiffeurs, file, capacite, affichage)) 
            id_client+=1 # passer au client suivant   

###################################################################################

def traitement_client(env, moy_coupe, coiffeurs, id_client, nom_coiffeurs, file, capacite, affichage):
    
    global nb_rejet
    global t_attente
    global charge
    global temps_service
    
    temps_arrivee=env.now  #enregistrer le temps d'arrivée du client
    if len(file)<capacite :
        file.append(id_client) #ajouter le client à la file d'attente
        if affichage==True : print("[%.2f] -> Arrivée du client %d  File:%s"%(temps_arrivee, id_client, file))
    else :
        if affichage==True : print("[%.2f] -> client %d rejeté car la file d'attente est pleine"%(temps_arrivee, id_client))
        nb_rejet = nb_rejet + 1
        return #sauter le traitement du client rejeté
    
    with coiffeurs.request() as req: #attendre qu'un coiffeur se libère (cet appel est bloquant)
        yield req #réserver le coiffeur
        temps_debut=env.now #enregistrer le temps de début de coupe
        coiff=nom_coiffeurs.pop(0) #trouver le nom du premier coiffeur libre
        if affichage==True : print("[%.2f] Début de coupe pour le client %d par %s"%(temps_debut,id_client, coiff))
        
        coupe=st.expon(0, moy_coupe).rvs() #simuler la durée du traitement par la loi exponentielle           
        yield env.timeout(coupe) #avancer l'horloge du simulateur jusqu'à la fin du traitement
        
        temps_fin=env.now #enregistrer le temps de fin de coupe
        if affichage==True : print("[%.2f] Fin de coupe pour le client %d ->"%(temps_fin,id_client))
        
        t_attente.append(temps_debut - temps_arrivee)
        
        file.remove(id_client) #supprimer le client de la file d'attente
        nom_coiffeurs.append(coiff) #remettre le nom du coiffeur dans la liste des serveurs libres
        
        # ajouter le temps de coupe au temps de coupe de ce coiffeur
        for i in range (len(nom_coiffeurs)) :
            if coiff == nom_coiffeurs[i]:
                temps_coupe[i] += coupe
                break
    
####################################################################################    

# 1.Appel de la fonction de simulation
print ("################## simulation ####################")

nom_coiffeurs=["Coiffeur0"]
duree=14400 # duree de la simulation 14.400 mintes
moy_client=5/60 # moyenne des clients arrivés par minute 
moy_coupe=10 # moyenne de durée de coupe
capacite=4 # capacité maximale de la file d'attente
affichage = False # si affichage vrai afficher les details de la simulation
salaire=0
prix=500
simulation(duree, moy_client, moy_coupe, nom_coiffeurs,capacite, affichage)
nom_coiffeurs=["Coiffeur0"]
metriques(nom_coiffeurs,duree,salaire,prix)

# 2.simulation pour capacité=6
print ("\n################## simulation pour capacité=6 ####################")

nom_coiffeurs=["Coiffeur0"]
capacite=6 # capacité maximale de la file d'attente
simulation(duree, moy_client, moy_coupe, nom_coiffeurs,capacite, affichage)
nom_coiffeurs=["Coiffeur0"]
metriques(nom_coiffeurs,duree,salaire,prix)
# il ne serait pas interessant d'augmenter le nombre de places dans la salle d'attente
# car en minimiser le taux de rejet, le temps d'attente augmente en parallele
# il faut en plus d'augmentation de capacité l'ajout d'autres coiffeurs 

# 3.simulation pour ajout d'un coiffeur
print ("\n################## simulation pour ajout d'un coiffeur ####################")

nom_coiffeurs=["Coiffeur0","Coiffeur1"]
salaire=70000
simulation(duree, moy_client, moy_coupe, nom_coiffeurs,capacite, affichage)
nom_coiffeurs=["Coiffeur0","Coiffeur1"]
metriques(nom_coiffeurs,duree,salaire,prix)
# oui c'est interessant car on a diminué le temps dattente et le taux de rejet
# par consequent augmenté le chiffre d'affaire
# le gain rapporté par l'ajout de ce coiffeur est superieur a la perte lors du paymenet des salaires

# 4.simulation pour ajout de deux coiffeurs
print ("\n##################### simulation pour ajout de deux coiffeurs ######################")

nom_coiffeurs=["Coiffeur0","Coiffeur1","Coiffeur2"]
simulation(duree, moy_client, moy_coupe, nom_coiffeurs,capacite, affichage)
nom_coiffeurs=["Coiffeur0","Coiffeur1","Coiffeur2"]
metriques(nom_coiffeurs,duree,salaire,prix)
# non car le gain rapporté par l'ajout de ce coiffeur est égal a la perte lors du paymenet des salaires

################## simulation ####################

[0.00] -------------- Début de la simulation --------------
[14400.00] -------------- Fin de la simulation ----------------

nombre de clients reçus:  1188
nombre de clients rejetés:  170
nombre de clients traités:  1018
taux de rejet: 14.31 %
temps d'attente moyen:  12.923817964339863
charge du Coiffeur0 : 71.52
Chiffre d'affaires total:  509000 DA

################## simulation pour capacité=6 ####################

[0.00] -------------- Début de la simulation --------------
[14400.00] -------------- Fin de la simulation ----------------

nombre de clients reçus:  1190
nombre de clients rejetés:  82
nombre de clients traités:  1108
taux de rejet: 6.89 %
temps d'attente moyen:  16.534583607339687
charge du Coiffeur0 : 149.09
Chiffre d'affaires total:  554000 DA

################## simulation pour ajout d'un coiffeur ####################

[0.00] -------------- Début de la simulation --------------
[14400.00] -------------- Fin de la si